In [8]:
# Question 1

# Llama-2-13B 
# L	40
# D	5,120
# F	13824
# N	40
# K	40
# H	128
# V	32,000

# Vocab
# 2DV

# Attention
# Q, K, V, O
# 4D^2

# MLP
# 3DF

V = 32000
D = 5120
F = 13824
L = 40

block_params =  4 * D**2 + 3 * D * F
total_params = 2 * V * D + block_params * L
print(f"total params: {total_params}")

total params: 13015449600


In [ ]:
# Question 2

#  B*S = 16M tokens
# model parameters: 2N
# model optimizer state: 4N + 4N (moment, second moment)
# roughly 13B * 10 = 130 GB
# model activations: (BF * 2 + BD)  * L * 2
        # where B = 16M
# roughly 2 * 40 * (16e6 * 13824 * 2 + 16e6 * 5120) = 42TB

# Activations take up most of the memory

In [ ]:
# Question 3

# Let P = model parameters and N be the number of chips in the slice
# 4096 chips, 16 x 16 x 16 topology
# First question is activation memory pressure:
# 95GB HBM
# Secnd question is 1. B / N > C / W_ici is the rule of thumb for data parallelism for compute bound

# Data Parallelism
# 1. Can't do pure data parallelism since the model + optimizer states will be too large for single chip

# FSDP
# 2. Imagine FSDP with the whole thing as the FSDP group
# Divide by the batch and divide by the optimizer states and params
# so params + optimizer states = 10P / N = 3e-2 GB
# activation is 2L (2BF + BD) / N = 1.9B
# The activations + params fit for FSDP
# Comm bound calculation: B / N > C / W_ici Mx (Mx in this case is 3 since 3 axes)
# C / W_ici is roughly 2550 so then B > 850 N which is ~3.48M tokens, we are comms bound here

# Tensor parallelism
# Yes, no reason why we can't?
# Following X_opt rule, X = sqrt(BNM_x F / M_y) = sqrt(3e6 * 2 * 4096 * / 13824) = ~1333
# Mx/My = 2  because we assume that the FSDP has two axes
# we make one axis TP = 4 and other two for FSDP = 1024
# so 1024-way FSDP and 4-way TP
# 3e-2 GB for params + optimizer states
# activations: 10P / N  + 2L * (2BF/Y + BD/Y) / X = 
# activations: also 1.9B? 
# T_math / T_comm  = sqrt(BF) sqrt(M_xM_y) / (alpha sqrt(N)) > 1
# sqrt(3e6 * 13824) * sqrt(2) / (2550 * sqrt(4096)) = 1.76 which means we are compute bound here!